*This notebook is part of our text recognition project for class CS345 at Colorado State University.
Original versions were created by Zachary Shimpa, Jenelle Dobyns and Jordan Rust.
The content is availabe [on GitHub](github.com/ZorkDaNerd/CS345-Text-Recognition).*

*Code help and referance was provided from Prof. Asa Ben-Hur and CS 345: Machine Learning Foundations and Practice at Colorado State University.
Original versions of these notebooks were created by Asa Ben-Hur with updates by Ross Beveridge.
The content is availabe [on his GitHub](https://github.com/asabenhur/CS345).*

<a href="https://colab.research.google.com/github/ZorkDaNerd/CS345-Text-Recognition/blob/main/Text_Recognition_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Possible data sets


https://www.kaggle.com/datasets/konradb/text-recognition-total-text-dataset


https://github.com/topics/text-recognition

https://analyticsindiamag.com/10-open-source-datasets-for-text-classification/

https://github.com/amephraim/nlp/tree/master/code 
Dataset from this

https://medium.com/mlearning-ai/sentiment-analysis-using-lstm-21767a130857

https://en.wikipedia.org/wiki/Sentiment_analysis



# Description of Project

This project is about recognizing text emotions using LSDM. This is a form of natural language processing.

### Coding languages and packages used in project

Ex: Anaconda, Python, ect

In [53]:
#Imports
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import os
from glob import glob
import tensorflow as tf
from tensorflow import keras
import string

# Project Code

In [55]:
#Defaults
books = []
thinBooks = []
uselessWords = ["the", "and", "a", "to", "of", "i", "in", "a", "there"]

#Putting all of the books into a np array for convinance
for file in glob('texts\*.txt'):
    books.append(open(file, 'r').read())
    thinBooks.append(open(file, 'r').read())
books = np.array(books)
thinBooks = np.array(thinBooks)

#Removes all of the punctuation and useless words then makes all of the words lowercase
for numb in range(len(thinBooks)):
    thinBooks[numb] = thinBooks[numb].translate(str.maketrans('', '', string.punctuation)).lower()
    editBook = thinBooks[numb].split()
    thinnedBook = [word for word in editBook if word not in uselessWords]
    thinBooks[numb] = ' '.join(thinnedBook) 
print(thinBooks[1])



harry potter chamber secrets by j k rowling this is book 2 harry potter series original scannedocr friday april 07 2000 v10 edit where needed change version number by 01 c h p t e rr o n e worst birthday not for first time an argument had broken out over breakfast at number four privet drive mr vernon dursley had been woken early hours morning by loud hooting noise from his nephew harrys room third time this week he roared across table if you cant control that owl itll have go harry tried yet again explain shes bored he said shes used flying around outside if could just let her out at night do look stupid snarled uncle vernon bit fried egg dangling from his bushy mustache know whatll happen if that owls let out he exchanged dark looks with his wife petunia harry tried argue back but his words were drowned by long loud belch from dursleys son dudley 1 want more bacon theres more frying pan sweetums said aunt petunia turning misty eyes on her massive son we must build you up while weve g

# Findings

# Analysis of results